In [1]:
# This notebook stores a list of unillustrated articles with suggested images in hdfs. See https://phabricator.wikimedia.org/T299789

import re
import math
import os
from wmfdata.spark import get_session

You are using wmfdata v1.3.1, but v1.3.2 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release --ignore-installed`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md


In [2]:
# Pass in the full snapshot date
snapshot = '2022-01-24'
reg = r'^([\w]+-[\w]+)'
short_snapshot = re.match(reg, snapshot).group()

In [3]:
# We use wmfdata boilerplate to init a spark session.
# Under the hood the library uses findspark to initialise
# Spark's environment. pyspark imports will be available 
# after initialisation
spark = get_session(type='regular', app_name="T299789")
import pyspark
import pyspark.sql

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [4]:
# load data stored by previous scripts

commons_file_pages = spark.read.parquet('hdfs:/user/mfossati/commons_file_pages')
commons_file_pages.createOrReplaceTempView('commons_file_pages')
commons_files_related_wikidata_items = spark.read.parquet('commons_files_related_wikidata_items')
commons_files_related_wikidata_items.createOrReplaceTempView('commons_files_related_wikidata_items')

In [5]:
query="""WITH suggestion_qid_p18 AS
(
 SELECT cfp.page_title, EXPLODE(cw.reverse_p18) as wikidataId, 'wikidata-P18' as source, NULL as found_on 
 FROM commons_files_related_wikidata_items cw
 JOIN commons_file_pages cfp
 ON cfp.page_id=cw.page_id
 WHERE cw.reverse_p18 IS NOT NULL
),
suggestion_qid_p373 AS
(
 SELECT page_title, SPLIT(info,"\\\|")[0] as wikidataId, 'wikidata-P373' as source, NULL as found_on
 FROM 
 (
     SELECT cfp.page_title as page_title, EXPLODE(cw.reverse_p373) as info
     FROM commons_files_related_wikidata_items cw
     JOIN commons_file_pages cfp
     ON cfp.page_id=cw.page_id
     WHERE cw.reverse_p373 IS NOT NULL
 )
),
suggestion_qid_leadImage AS
(
 SELECT page_title, 
 SPLIT(info,"\\\|")[0] as wikidataId,
 'page_props-page_image_free' as source, 
 collect_set(SPLIT(info,"\\\|")[1]) as found_on
 FROM 
 (
     SELECT cfp.page_title as page_title, EXPLODE(cw.container_page_qids) as info
     FROM commons_files_related_wikidata_items cw
     JOIN commons_file_pages cfp
     ON cfp.page_id=cw.page_id
     WHERE cw.container_page_qids IS NOT NULL
 )
 GROUP BY page_title, wikidataId
),
commons_statements AS 
( 
 SELECT id AS mId,
 EXPLODE(statements) AS statement 
 FROM structured_data.commons_entity WHERE snapshot='"""+snapshot+"""' 
),
suggestion_qid_commons AS
(
 SELECT DISTINCT from_json(statement.mainsnak.datavalue.value, 'entityType STRING, numericId INT, id STRING').id AS wikidataId,
 cfp.page_title,
 concat('commons-', statement.mainsnak.property) as source,
 NULL as found_on
 FROM commons_statements cs
 JOIN commons_file_pages cfp 
 ON cfp.page_id=SUBSTRING( cs.mId, 2 )
 WHERE statement.mainsnak.property IN ('P180', 'P6243')
)
SELECT wikidataId, page_title AS suggestion, source, found_on FROM suggestion_qid_p18
UNION 
SELECT wikidataId, page_title AS suggestion, source, found_on FROM suggestion_qid_p373
UNION
SELECT wikidataId, page_title AS suggestion, source, found_on FROM suggestion_qid_leadImage
UNION
SELECT wikidataId, page_title AS suggestion, source, found_on FROM suggestion_qid_commons
"""
wdSuggestionsDF = spark.sql(query)
wdSuggestionsDF.createOrReplaceTempView("all_suggestions")

In [6]:
# articles corresponding to wikidata items that are instances of lists, or years, or names, etc should NOT be illustrated, so filter out suggestions associated with those items

unillustratable = [
    "Q577", # year
    "Q29964144", # calendar year
    "Q14795564", # recurrent timeframe
    "Q3311614", # century leap year
    "Q101352", # family name
    "Q82799", # name
    "Q4167410", # list
    "Q21199", # natural number
    "Q28920044", # positive integer
    "Q28920052", # non negative integer
]
query = """
SELECT wikidataId, suggestion, 
collect_set(source) AS sources, found_on, 
collect_set(from_json(claim.mainSnak.dataValue.value, 'entityType STRING, numericId INT, id STRING').id) as instance_of
FROM all_suggestions as
JOIN wmf.wikidata_entity we 
ON as.wikidataId=we.id
LATERAL VIEW OUTER explode(we.claims) c AS claim
WHERE we.typ='item'
AND claim.mainSnak.property='P31'
AND we.snapshot='"""+snapshot+"""'
AND from_json(claim.mainSnak.dataValue.value, 'entityType STRING, numericId INT, id STRING').id NOT IN ('""" + "','".join(unillustratable) + """')
GROUP BY wikidataId,suggestion,found_on
ORDER BY wikidataId,suggestion
"""
fsDF = spark.sql(query)
fsDF.createOrReplaceTempView("filtered_suggestions")

In [7]:
languages=['ptwiki']
#,'ruwiki']
for wiki in languages:
    query = """WITH illustrated_pages
    (
        SELECT DISTINCT il_from
        FROM wmf_raw.mediawiki_imagelinks il
        JOIN commons_file_pages cfp
        ON cfp.page_title=il.il_to
        WHERE il.wiki_db='"""+wiki+"""' 
        AND il.snapshot='"""+short_snapshot+"""' 
    )
    SELECT p.wiki_db,p.page_id,p.page_title,fs.suggestion,fs.sources,fs.found_on,fs.instance_of
    FROM wmf_raw.mediawiki_page p
    LEFT ANTI JOIN illustrated_pages il
    ON il.il_from=p.page_id
    JOIN wmf.wikidata_item_page_link wipl
    ON p.page_id=wipl.page_id
    JOIN filtered_suggestions fs
    ON wipl.item_id=fs.wikidataId
    WHERE p.page_namespace=0 
    AND page_is_redirect=0 
    AND p.wiki_db='"""+wiki+"""' 
    AND p.snapshot='"""+short_snapshot+"""'
    AND wipl.wiki_db='"""+wiki+"""'
    AND wipl.snapshot='"""+snapshot+"""'
    """
    suggestionsDF = spark.sql(query)
    suggestionsDF.show(50,truncate=False)

+-------+-------+------------------------------------+------------------------------------------------------------------------------------------------+-----------------------------+-----------------------------------------------------------------+--------------------------------+
|wiki_db|page_id|page_title                          |suggestion                                                                                      |sources                      |found_on                                                         |instance_of                     |
+-------+-------+------------------------------------+------------------------------------------------------------------------------------------------+-----------------------------+-----------------------------------------------------------------+--------------------------------+
|ptwiki |1362396|Tintah_(Minnesota)                  |Traverse_County_Minnesota_Incorporated_and_Unincorporated_areas_Tintah_Highlighted.svg          |[page_